In [124]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import operator
import scipy.spatial.distance as distance
import sklearn.metrics
import random
import time

In [20]:
df = pd.read_pickle('processed_df.pkl')
ratings = pd.read_pickle('ratings_sample.pkl')
ratings = ratings.reset_index()

In [21]:
movies = df.drop(columns = ['movieId', 'title_eng', 'year'])

## Build Recommendations for 1 User
To Do:
- Extend to all users (function) -done
- Test/train split so hold out some recommendations to evaluate against --> evaluate

Extensions:
- Include year (decade)?
- Include text from description or genome tags
- Downweight older ratings 

In [22]:
# limit to one user
ratings_user = ratings[ratings.userId == 100]
movies_user = df[df.movieId.isin(ratings_user.movieId.unique())]

# keep track of movies already watched
watched = movies_user.movieId.unique()

movies_user = movies_user.drop(columns = ['movieId', 'title_eng', 'year'])

In [23]:
# normalize ratings: subtract mean rating  from ratings
# if rating < mean, normalized rating will be negative. Which is worse than 0 aka not rating movie at all.

#### should this be averaged??

mean_rating = np.mean(ratings_user.rating)
ratings_user.rating = ratings_user.rating - mean_rating

/opt/anaconda3/envs/Peritia1/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
# create user profile: multiply item profile by user ratings --> sum of ratings for each attribute 
profile = movies_user.T.dot(ratings_user.rating.values)

In [25]:
# cosine similarity between movie and user profile 
movies = df.drop(columns = ['movieId', 'title_eng', 'year'])
recommendations = sklearn.metrics.pairwise.cosine_similarity(movies, np.asmatrix(profile.values))

In [26]:
recommendations = pd.DataFrame(recommendations)
recommendations.columns = ['prediction']

# merge to get title
recommendations = pd.merge(recommendations, df[['title_eng', 'movieId']], left_index = True, right_index = True, how = 'left')

# sort 
recommendations = recommendations.sort_values('prediction', ascending = False)

In [27]:
# remove recommendations already watched
recommendations = recommendations[~recommendations.movieId.isin(watched)]

In [28]:
# movies with the same prediction -- should we secondarily sort by overall popularity of the movie?
recommendations[:10]

,prediction,title_eng,movieId
39994,0.512362,In August of 1944 (2001),188375
41879,0.512362,Deadline (1987),195153
1235,0.459565,"Forbidden Christ, The (Cristo proibito, Il) (1...",1368
24720,0.416294,Ground Zero (1987),134268
32214,0.416294,Deo pa-i-beu (2013),160654
26212,0.416294,Crimes of the Past (2009),139657
27711,0.416294,Broken (2014),144510
33375,0.397995,Line Walker (2016),164407
24151,0.397995,N.H 10 (2015),132358
9185,0.397995,Electra Glide in Blue (1973),32632


In [29]:
def user_content_recommendations(user_id):   
    """
    ratings_user: limit to one user
    
    movies_user: movies rated by that user
    
    watched: keep track of movies already watched
    
    normalize ratings: subtract mean rating  from ratings
                       if rating < mean, normalized rating will be negative. Which is worse than 0 aka not rating movie at all.
    
    profile:create user profile: multiply item profile by user ratings --> sum of ratings for each attribute 
    
    recommendations: cosine similarity between movie and user profile 
                     merge to get title
                     sort
                     remove recommendations already watched
    """

    ratings_user = ratings[ratings.userId == user_id]
    movies_user = df[df.movieId.isin(ratings_user.movieId.unique())]
    watched = movies_user.movieId.unique()
    movies_user = movies_user.drop(columns = ['movieId', 'title_eng', 'year'])
    
    mean_rating = np.mean(ratings_user.rating)
    ratings_user.rating = ratings_user.rating - mean_rating

    profile = movies_user.T.dot(ratings_user.rating.values)
    
    movies = df.drop(columns = ['movieId', 'title_eng', 'year'])
   
    recommendations = sklearn.metrics.pairwise.cosine_similarity(movies, np.asmatrix(profile.values))
    recommendations = pd.DataFrame(recommendations)
    recommendations.columns = ['prediction']
    recommendations = pd.merge(recommendations, df[['title_eng', 'movieId']], left_index = True, right_index = True, how = 'left')
    recommendations = recommendations.sort_values('prediction', ascending = False)
    recommendations = recommendations[~recommendations.movieId.isin(watched)]
    return recommendations

def user_top(movies,n):
    return movies['title_eng'][:n]

def user_movie_id(movies,n):
    return movies['movieId'][:n]

### Getting top 10 and top 5 recommendation for user_id = 100

In [30]:
userId=100
recommendation = user_content_recommendations(userId)

/opt/anaconda3/envs/Peritia1/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [31]:
# Get the top 10 Movie recommendation for the user
user_top(recommendation,10)

39994                             In August of 1944 (2001)
41879                                      Deadline (1987)
1235     Forbidden Christ, The (Cristo proibito, Il) (1...
24720                                   Ground Zero (1987)
32214                                  Deo pa-i-beu (2013)
26212                            Crimes of the Past (2009)
27711                                        Broken (2014)
33375                                   Line Walker (2016)
24151                                        N.H 10 (2015)
9185                          Electra Glide in Blue (1973)
Name: title_eng, dtype: object

In [32]:
# Get the top 5 Movie recommendation for the user
user_top(recommendation,5)

39994                             In August of 1944 (2001)
41879                                      Deadline (1987)
1235     Forbidden Christ, The (Cristo proibito, Il) (1...
24720                                   Ground Zero (1987)
32214                                  Deo pa-i-beu (2013)
Name: title_eng, dtype: object

## Evaluation: Personalization



In [125]:
"""
get_users_prediction: returns user prediction

user_matrix: generated user by movies, 0: movie not recommended, 1: movie got recommended

"""
def get_users_prediction(users_list,top_n,users_prediction):
    for i in users_list:
        recommendation = user_content_recommendations(i)
        prediction = user_movie_id(recommendation,top_n).astype(int).values
        users_prediction = users_prediction.append(pd.Series(prediction),ignore_index=True)
        
    return users_prediction

def user_matrix(df):
    data_melt= pd.DataFrame(data=users_pred).reset_index().melt(id_vars='index', value_name='movieId',)
    data_melt = data_melt[['index', 'movieId']].pivot(index='index', columns='movieId', values='movieId')
    cols = data_melt.columns
    for i in cols:
        data_melt[i] = np.where(data_melt[i].isna(), 0, 1)
    return data_melt


def personalization(users_matrix,n):
    # generating cosine similarity between the users
    #then getting the indicies of elements above diagonal (giving diagonal offsert =1 in triu_indices)
    #calculating the avg of the element above diagonal 
    #Personalization means 1 - similarity
    #higher the personalization score, better the recommendation system in recommending personalized movies
   
    users_sim = sklearn.metrics.pairwise.cosine_similarity(users_matrix)
    iu1 = np.triu_indices(n,k=1)
    similarity_avg = np.mean(users_sim[iu1])
    personalization_score = 1 - similarity_avg

    return personalization_score

def cross_fold_eval(unique_users,k_fold=5,n=5,top_n=10):
    #users_list: List of random n users
    
    kfold_personalization=0
    for i in range(k_fold):
        users_list = random.sample(unique_users, n)
        
        #columns: top_n recommendations
        #users_prediction:  top_n recommendations for n users
        column_names = list(range(top_n))
        users_prediction = pd.DataFrame(columns = column_names)
        
        # getting predictions for sampled users
        users_pred = get_users_prediction(users_list,top_n,users_prediction)
        
        # getting user by movies matrix with binary indicators 0: movie not recommended, 1: movie got recommended
        users_matrix = user_matrix(users_pred)
        kfold_personalization+=personalization(users_matrix,n)
    kfold_eval = kfold_personalization/k_fold
    print(f'Personalization score for {k_fold} k_fols validation: {kfold_eval}')
        

In [126]:
"""
    unique_users: getting unique user set 
    
    default values given in function
    n: number of users (5)
    top_n: top number of recommendations (10)
"""

unique_users = set(ratings.userId)
start_time = time.time()
cross_fold_eval(unique_users)

end_time = time.time() - start_time
print(f'Time take {end_time} seconds')
    
    





/opt/anaconda3/envs/Peritia1/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/anaconda3/envs/Peritia1/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/anaconda3/envs/Peritia1/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

Personalization score for 2 k_fols validation: 1.0
Time take 207.35414505004883 seconds


## Evaluation: Diversity

### Approach 1: Intra List Similarity

### Approach 2: Content diversity